In [2]:
import h5py
import pandas as pd
import json
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ssp
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from collections import Counter
import pickle
from sklearn.utils import shuffle
from os.path import join as pjoin
from tqdm.auto import tqdm
import os
import joblib
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import importlib 
from sklearn.preprocessing import normalize as sk_normalize
import pytorch_lightning as pl
from os.path import join as pjoin
from scipy import sparse

In [3]:
def build_dataset(sessions, purchases=None):
    item_feats = pd.read_csv('../data/item_features.csv', dtype=str)
    item_feats['feat_id'] = item_feats['feature_category_id'] + ':' + item_feats['feature_value_id']
    d = item_feats.groupby("item_id")['feat_id'].apply(list).reset_index()
    item_to_feats = {x:y for (x, y) in zip(d.item_id, d.feat_id)}

    def items_to_feats(item_ids, item_to_feats):
        ret = []
        for x in item_ids:
            ret.extend(item_to_feats[x])
        return ret

    sessions['kp'] = [(x,y) for (x,y) in zip(sessions.date, sessions.item_id)]
    t = sessions.groupby('session_id').kp.apply(lambda x: [y for y in sorted(x)]).reset_index()
    q = sessions.groupby('session_id')['date'].apply(lambda x: list(sorted(x))[0])
    v = sessions.groupby('session_id')['date'].apply(lambda x: list(sorted(x))[-1])
    q = pd.merge(q, v, on='session_id')
    print(t.shape, q.shape,v.shape)
    ret = pd.merge(pd.DataFrame(t), pd.DataFrame(q), on='session_id')
    if purchases is not None:
        dataset = pd.merge(ret, purchases, on='session_id')
        dataset.columns = ['session_id', 'item_ids', 'date_session_begin', 'date_session_end', 'target_item', 'date_purchase']
        dataset['target_feat'] = dataset.target_item.apply(lambda x: item_to_feats[x])
    else:
        dataset = ret
        dataset.columns = ['session_id', 'item_ids', 'date_session_begin', 'date_session_end']

    dataset['feats'] = dataset.item_ids.apply(lambda x: items_to_feats([y[1] for y in x], item_to_feats))
    dataset['ym'] = dataset.date_session_end.apply(lambda x:x[:7])
    dataset['year'] = dataset.date_session_end.apply(lambda x:x[:4])
    dataset['month'] = dataset.date_session_end.apply(lambda x:x[5:7])
    dataset['dow'] =  pd.to_datetime(dataset.date_session_end).dt.dayofweek.apply(lambda x: str(x))
    dataset.sort_values(by='date_session_end', inplace=True)
    return dataset

In [4]:
train_sessions = pd.read_csv('../data/train_sessions.csv', dtype=str)
train_purchases = pd.read_csv('../data/train_purchases.csv', dtype=str)
train_sessions = train_sessions[train_sessions.date >= '2021-02-01 00:00:00.000']

In [5]:
leaderboard_sessions = pd.read_csv('../data/test_leaderboard_sessions.csv', dtype=str)

In [6]:
final_sessions = pd.read_csv('../data/test_final_sessions.csv', dtype=str)

In [7]:
_tr_csv = build_dataset(train_sessions, train_purchases).sort_values(by='date_session_end').reset_index(drop=True)

(259984, 2) (259984, 2) (259984,)


In [8]:
val_csv = _tr_csv[-3000:]

In [9]:
leaderboard_csv = build_dataset(leaderboard_sessions)

(50000, 2) (50000, 2) (50000,)


In [10]:
final_csv = build_dataset(final_sessions)

(50000, 2) (50000, 2) (50000,)


In [11]:
tr2 = _tr_csv.copy()

In [12]:
tr2.head(3)

,session_id,item_ids,date_session_begin,date_session_end,target_item,date_purchase,target_feat,feats,ym,year,month,dow
0,3308053,"[(2021-02-01 00:04:49.941, 19882), (2021-02-01...",2021-02-01 00:04:49.941,2021-02-01 00:06:32.387,6229,2021-02-01 00:06:57.192,"[56:365, 26:268, 59:387, 17:378, 4:618, 61:706...","[56:365, 61:462, 33:36, 53:444, 24:588, 29:123...",2021-02,2021,02,0
1,751312,"[(2021-02-01 00:06:53.005, 3558), (2021-02-01 ...",2021-02-01 00:06:53.005,2021-02-01 00:07:29.192,13096,2021-02-01 07:33:49.638,"[56:365, 57:135, 5:605, 70:134, 72:75, 61:462,...","[56:365, 18:817, 46:825, 3:793, 47:549, 63:861...",2021-02,2021,02,0
2,347839,"[(2021-02-01 00:07:42.591, 9184)]",2021-02-01 00:07:42.591,2021-02-01 00:07:42.591,18156,2021-02-01 00:08:16.102,"[24:588, 56:365, 50:240, 62:801, 47:123, 28:43...","[56:365, 47:123, 69:885, 72:75, 73:544, 7:394,...",2021-02,2021,02,0


In [14]:
tr2['target_item'] = tr2['item_ids'].apply(lambda x:x[-1][1])

In [15]:
tr2['item_ids'] = tr2['item_ids'].apply(lambda x:x[0:-1])

In [16]:
tr2 = tr2[tr2.item_ids.apply(len) > 0]

In [17]:
q = [tr2.reset_index(drop=True), _tr_csv.reset_index(drop=True)]
tr_csv = pd.concat(q, axis=0)

In [18]:
tr_csv = tr_csv.sort_values(by='date_session_end').reset_index(drop=True)

In [19]:
def flatten(t):
    return [item for sublist in t for item in sublist]
item_feats = pd.read_csv('../data/item_features.csv', dtype=str)
item_feats['feat_id'] = item_feats['feature_category_id'] + ':' + item_feats['feature_value_id']
d = item_feats.groupby("item_id")['feat_id'].apply(list).reset_index()
item_to_feats = {x:y for (x, y) in zip(d.item_id, d.feat_id)}

In [20]:
# tr_csv = _tr_csv.reset_index(drop=True)
leaderboard_csv = leaderboard_csv.reset_index(drop=True)
final_csv = final_csv.reset_index(drop=True)
val_csv = val_csv.reset_index(drop=True)

In [21]:
val_dur = (pd.to_datetime(val_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(val_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds
tr_dur = (pd.to_datetime(tr_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(tr_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds
leaderboard_dur = (pd.to_datetime(leaderboard_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(leaderboard_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds
final_dur = (pd.to_datetime(final_csv.item_ids.apply(lambda x: x[-1][0])) - pd.to_datetime(final_csv.item_ids.apply(lambda x: x[0][0]))).dt.seconds

In [22]:
val_csv['item_ids'] = val_csv.item_ids.apply(lambda x : [y[1] for y in x])
tr_csv['item_ids'] = tr_csv.item_ids.apply(lambda x : [y[1] for y in x])
leaderboard_csv['item_ids'] = leaderboard_csv.item_ids.apply(lambda x : [y[1] for y in x])
final_csv['item_ids'] = final_csv.item_ids.apply(lambda x : [y[1] for y in x])

In [23]:
val_l = val_csv.item_ids.apply(len)
tr_l = tr_csv.item_ids.apply(len)
leaderboard_l = leaderboard_csv.item_ids.apply(len)
final_l = final_csv.item_ids.apply(len)

In [24]:
v_pop_all = Counter(flatten(tr_csv.item_ids.tolist()))
p_pop_all = Counter(tr_csv.target_item.tolist())

In [25]:
tr_v_pop_all = tr_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
tr_p_pop_all = tr_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])


val_v_pop_all = val_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
val_p_pop_all = val_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

leaderboard_v_pop_all = leaderboard_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
leaderboard_p_pop_all = leaderboard_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

final_v_pop_all = final_csv.item_ids.apply(lambda x: [v_pop_all[y] for y in x])
final_p_pop_all = final_csv.item_ids.apply(lambda x: [p_pop_all[y] for y in x])

In [26]:
v_pop_3m = Counter(flatten(tr_csv[tr_csv.ym >= "2021-05"].item_ids.tolist()))
p_pop_3m = Counter(tr_csv[tr_csv.ym >= "2021-05"].target_item.tolist())

In [27]:
tr_v_pop_3m = tr_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
tr_p_pop_3m = tr_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

val_v_pop_3m = val_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
val_p_pop_3m = val_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

leaderboard_v_pop_3m = leaderboard_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
leaderboard_p_pop_3m = leaderboard_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

final_v_pop_3m = final_csv.item_ids.apply(lambda x: [v_pop_3m[y] for y in x])
final_p_pop_3m = final_csv.item_ids.apply(lambda x: [p_pop_3m[y] for y in x])

In [28]:
val_s_len = val_csv.item_ids.apply(len)
tr_s_len = tr_csv.item_ids.apply(len)
leaderboard_s_len = leaderboard_csv.item_ids.apply(len)
final_s_len = final_csv.item_ids.apply(len)

In [29]:
tr_tmp = pd.concat([
    np.log(1 + tr_s_len),
    np.log(1 + tr_dur),
    np.log(1 + tr_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + tr_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + tr_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + tr_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + tr_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + tr_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + tr_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + tr_p_pop_3m.apply(lambda x: np.max(x))),
    
], axis=1)

val_tmp = pd.concat([
    np.log(1 + val_s_len),
    np.log(1 + val_dur),
    np.log(1 + val_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + val_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + val_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + val_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + val_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + val_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + val_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + val_p_pop_3m.apply(lambda x: np.max(x))),
    
], axis=1)

leaderboard_tmp = pd.concat([
   np.log(1 + leaderboard_s_len),
    np.log(1 + leaderboard_dur),
    np.log(1 + leaderboard_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + leaderboard_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + leaderboard_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + leaderboard_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + leaderboard_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + leaderboard_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + leaderboard_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + leaderboard_p_pop_3m.apply(lambda x: np.max(x))),
], axis=1)

final_tmp = pd.concat([
   np.log(1 + final_s_len),
    np.log(1 + final_dur),
    np.log(1 + final_v_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + final_p_pop_all.apply(lambda x: np.mean(x))),
    np.log(1 + final_v_pop_all.apply(lambda x: np.max(x))),
    np.log(1 + final_p_pop_all.apply(lambda x: np.max(x))),
    
    np.log(1 + final_v_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + final_p_pop_3m.apply(lambda x: np.mean(x))),
    np.log(1 + final_v_pop_3m.apply(lambda x: np.max(x))),
    np.log(1 + final_p_pop_3m.apply(lambda x: np.max(x))),
], axis=1)

tr_scalar = tr_tmp.to_numpy()
leaderboard_scalar = leaderboard_tmp.to_numpy()
final_scalar = final_tmp.to_numpy()

In [30]:
val_scalar = val_tmp.to_numpy()

In [31]:
def iddf(x):
    return x

In [32]:
tr_csv['tt'] = tr_csv.date_session_end.apply(lambda x: x[:9]) + ':' + tr_csv.item_ids.apply(lambda x: ':'.join(sorted(x[-5:])))

In [33]:
kw = tr_csv.groupby('tt').target_item.apply(list).reset_index()

In [34]:
mask = {x: y for (x, y) in zip(kw.tt, kw.target_item)}

In [35]:
input_mask_all = tr_csv.item_ids# + tr_csv.tt.apply(lambda x: mask[x])

In [36]:
ret = []
for x, y in  zip(input_mask_all, tr_csv.target_item):
    r = [k for k in x if k != y]
    ret.append(r)

In [37]:
input_mask_all = ret

In [38]:
lm_item = sorted(list(set(flatten(tr_csv.item_ids.apply(lambda x: x))) | set(tr_csv.target_item.unique())), key=lambda x: int(x))
# lm_item = set(item_to_feats.keys())
id2idx = {x:i for i,x in enumerate(lm_item)}
item_CV = CountVectorizer(analyzer =iddf, vocabulary=id2idx)

In [39]:
tr_mask_all = item_CV.transform(input_mask_all)

In [40]:
AA = tr_mask_all.toarray().sum(-1).ravel()

In [41]:
BB = item_CV.transform(tr_csv.item_ids).toarray().sum(-1).ravel()

In [42]:
(AA - BB).mean()

-0.11897727834810488

In [43]:
tr_targets = tr_csv.target_item.apply(lambda x: id2idx[x]).tolist()
val_targets = val_csv.target_item.apply(lambda x: id2idx[x]).tolist()
leaderboard_targets = [29999 for _ in range(len(leaderboard_csv))]
final_targets = [29999 for _ in range(len(final_csv))]
(np.array(leaderboard_targets) == 29999).sum(), (np.array(final_targets) == 29999).sum()

(50000, 50000)

In [44]:
item_all = item_CV.transform(tr_csv.item_ids.apply(lambda x: x))
last_20 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-20:]))
last_5 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-5:]))
last_2 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-2:]))
last_1 = item_CV.transform(tr_csv.item_ids.apply(lambda x: x[-1:]))

val_all = item_CV.transform(val_csv.item_ids.apply(lambda x: x))
val_last_20 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-20:]))
val_last5 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-5:]))
val_last2 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-2:]))
val_last1 = item_CV.transform(val_csv.item_ids.apply(lambda x: x[-1:]))


leaderboard_all = item_CV.transform(leaderboard_csv.item_ids.apply(lambda x: x))
leaderboard_last_20 = item_CV.transform(leaderboard_csv.item_ids.apply(lambda x: x[-20:]))
leaderboard_last5 = item_CV.transform(leaderboard_csv.item_ids.apply(lambda x: x[-5:]))
leaderboard_last2 = item_CV.transform(leaderboard_csv.item_ids.apply(lambda x: x[-2:]))
leaderboard_last1 = item_CV.transform(leaderboard_csv.item_ids.apply(lambda x: x[-1:]))

final_all = item_CV.transform(final_csv.item_ids.apply(lambda x: x))
final_last_20 = item_CV.transform(final_csv.item_ids.apply(lambda x: x[-20:]))
final_last5 = item_CV.transform(final_csv.item_ids.apply(lambda x: x[-5:]))
final_last2 = item_CV.transform(final_csv.item_ids.apply(lambda x: x[-2:]))
final_last1 = item_CV.transform(final_csv.item_ids.apply(lambda x: x[-1:]))

In [45]:
tr_last_5_feats = tr_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
tr_last_3_feats = tr_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
tr_last_1_feats = tr_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))

val_last_5_feats = val_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
val_last_3_feats = val_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
val_last_1_feats = val_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))


leaderboard_last_5_feats = leaderboard_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
leaderboard_last_3_feats = leaderboard_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
leaderboard_last_1_feats = leaderboard_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))

final_last_5_feats = final_csv.item_ids.apply(lambda x: x[-5:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
final_last_3_feats = final_csv.item_ids.apply(lambda x: x[-3:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))
final_last_1_feats = final_csv.item_ids.apply(lambda x: x[-1:]).apply(lambda x: flatten([item_to_feats[y] for y in x]))

In [46]:
year_map = {y: x for (x, y) in enumerate(['2020', '2021'])}
month_map = {y: x for (x, y) in enumerate(["%02d" % x for x in range(1, 13)])}
hour_map = {y: x for (x, y) in enumerate(["%d" % x for x in range(0, 24)])}
dow_map = {y: x for (x, y) in enumerate(tr_csv.dow.unique())}
tr_csv['dow'] = pd.to_datetime(tr_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
tr_csv['hour'] = pd.to_datetime(tr_csv.date_session_end).dt.hour.apply(lambda x: str(x))

val_csv['dow'] = pd.to_datetime(val_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
val_csv['hour'] = pd.to_datetime(val_csv.date_session_end).dt.hour.apply(lambda x: str(x))

leaderboard_csv['dow'] = pd.to_datetime(leaderboard_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
leaderboard_csv['hour'] = pd.to_datetime(leaderboard_csv.date_session_end).dt.hour.apply(lambda x: str(x))

final_csv['dow'] = pd.to_datetime(final_csv.date_session_end).dt.dayofweek.apply(lambda x: str(x))
final_csv['hour'] = pd.to_datetime(final_csv.date_session_end).dt.hour.apply(lambda x: str(x))

In [47]:
all_feats = flatten([item_to_feats[x] for x in lm_item])

In [48]:
all_feats = set(all_feats)

In [49]:
all_feats = set(flatten(item_to_feats.values()))

In [50]:
i2fidx = item_to_feats.keys()
feat_CV = CountVectorizer(analyzer=iddf, 
                          vocabulary=all_feats)

In [51]:
lm_feats = normalize(feat_CV.transform([item_to_feats[x] for x in lm_item]), norm='l1')

In [52]:
tr_l5_feats = normalize(feat_CV.transform(tr_last_5_feats), norm='l1').toarray()
tr_l3_feats = normalize(feat_CV.transform(tr_last_3_feats), norm='l1').toarray()
tr_l1_feats = normalize(feat_CV.transform(tr_last_1_feats), norm='l1').toarray()

val_l5_feats = normalize(feat_CV.transform(val_last_5_feats), norm='l1').toarray()
val_l3_feats = normalize(feat_CV.transform(val_last_3_feats), norm='l1').toarray()
val_l1_feats = normalize(feat_CV.transform(val_last_1_feats), norm='l1').toarray()

leaderboard_l5_feats = normalize(feat_CV.transform(leaderboard_last_5_feats), norm='l1').toarray()
leaderboard_l3_feats = normalize(feat_CV.transform(leaderboard_last_3_feats), norm='l1').toarray()
leaderboard_l1_feats = normalize(feat_CV.transform(leaderboard_last_1_feats), norm='l1').toarray()

final_l5_feats = normalize(feat_CV.transform(final_last_5_feats), norm='l1').toarray()
final_l3_feats = normalize(feat_CV.transform(final_last_3_feats), norm='l1').toarray()
final_l1_feats = normalize(feat_CV.transform(final_last_1_feats), norm='l1').toarray()

In [53]:
EMBED_DIM = 16
tr_cats = {
#     "year": (tr_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (tr_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (tr_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (tr_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),
}

val_cats = {
    #     "year": (val_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
    #     "month": (val_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
        "dow": (val_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
        "hour": (val_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),
    }

leaderboard_cats = {
#     "year": (leaderboard_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (leaderboard_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (leaderboard_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (leaderboard_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),    
}

final_cats = {
#     "year": (final_csv['year'].apply(lambda x: year_map[x]).tolist(), (2, EMBED_DIM)),
#     "month": (final_csv['month'].apply(lambda x: month_map[x]).tolist(), (12, EMBED_DIM)),
    "dow": (final_csv['dow'].apply(lambda x: dow_map[x]).tolist(), (7, EMBED_DIM)),
    "hour": (final_csv['hour'].apply(lambda x: hour_map[x]).tolist(), (25, EMBED_DIM)),    
}

In [54]:
tr_mats = {
    "item_bow": item_all, 
    "item_last_20": last_20,
    "item_last_5": last_5,
    "item_last_2": last_2,
    "item_last_1": last_1, 
    'item_last_5_feats': tr_l5_feats,
    "item_last_3_feats" : tr_l3_feats,
    "item_last_1_feats" : tr_l1_feats, 
}



val_mats  = {
    "item_bow": val_all, 
    "item_last_20": val_last_20,
    "item_last_5": val_last5,
    "item_last_2": val_last2,
    "item_last_1": val_last1, 
    'item_last_5_feats': val_l5_feats,
    "item_last_3_feats" : val_l3_feats,
    "item_last_1_feats" : val_l1_feats
}


leaderboard_mats  = {
    "item_bow": leaderboard_all, 
    "item_last_20": leaderboard_last_20,
    "item_last_5": leaderboard_last5,
    "item_last_2": leaderboard_last2,
    "item_last_1": leaderboard_last1, 
    'item_last_5_feats': leaderboard_l5_feats,
    "item_last_3_feats" : leaderboard_l3_feats,
    "item_last_1_feats" : leaderboard_l1_feats
}


final_mats  = {
    "item_bow": final_all, 
    "item_last_20": final_last_20,
    "item_last_5": final_last5,
    "item_last_2": final_last2,
    "item_last_1": final_last1, 
    'item_last_5_feats': final_l5_feats,
    "item_last_3_feats" : final_l3_feats,
    "item_last_1_feats" : final_l1_feats, 
}

In [55]:
import mlps
importlib.reload(mlps)

<module 'mlps' from '/data/private/recsys2022/mlp/mlps.py'>

In [56]:
val_mask_all = item_CV.transform(val_csv.item_ids)

In [57]:
leaderboard_mask_all = item_CV.transform(leaderboard_csv.item_ids)
final_mask_all = item_CV.transform(final_csv.item_ids)

In [58]:
tr_ds = mlps.dictDataset(tr_mask_all, tr_mats, tr_cats, tr_scalar, tr_targets)
val_ds = mlps.dictDataset(val_mask_all, val_mats, val_cats, val_scalar, val_targets)
leaderboard_ds = mlps.dictDataset(leaderboard_mask_all, leaderboard_mats, {}, leaderboard_scalar, leaderboard_targets)
final_ds = mlps.dictDataset(final_mask_all, final_mats, {}, final_scalar, final_targets)

In [60]:
fname = 'submit_dataset'

In [61]:
!rm -rf  ./submit_dataset
!mkdir submit_dataset

In [62]:
joblib.dump(lm_feats, '%s/lm_feats' % fname)
joblib.dump(tr_ds, "%s/tr_ds" % fname)
joblib.dump(val_ds, "%s/val_ds" % fname)
joblib.dump(leaderboard_ds, '%s/leaderboard_ds' % fname)
joblib.dump(final_ds, '%s/final_ds' % fname)
joblib.dump(final_csv, "%s/final_csv" % fname)
joblib.dump(leaderboard_csv, "%s/leaderboard_csv" % fname)
joblib.dump(val_csv, "%s/val_csv" % fname)
joblib.dump(tr_csv, '%s/tr_csv' % fname)
joblib.dump(id2idx, '%s/id2idx' % fname )
joblib.dump((item_CV, feat_CV), "%s/cv" % fname)

['submit_dataset/cv']